In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures

train = pd.read_csv('/content/sales_train.csv')
test = pd.read_csv('/content/test.csv')
shops = pd.read_csv('/content/shops.csv')
items = pd.read_csv('/content/items.csv')
item_categories = pd.read_csv('/content/item_categories.csv')

Общие продажи магазинов

In [2]:
average_sales_shops = train.groupby(by='shop_id')['item_cnt_day'].sum().to_frame()

Общие продажи товаров

In [3]:
average_sales_items = train.groupby(by='item_id')['item_cnt_day'].sum()

Добавление пропущенных значений(нулей)

In [4]:
updated_train = train.groupby(by=['shop_id', 'item_id'])['item_cnt_day'].sum()
updated_train = updated_train.to_frame()
updated_train = updated_train.reset_index()
updated_train['key'] = 1
zeros = pd.DataFrame(range(34))
zeros.columns = ['date_block_num']
zeros['key'] = 1
updated_train = updated_train.merge(zeros, how='outer', on='key').drop('key', 1)
updated_train['item_cnt_day'] = 0
updated_train = updated_train.groupby(by=['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum()
new = train.groupby(by=['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum()
updated_train = new.add(updated_train, fill_value=0)
updated_train = updated_train.to_frame()
updated_train = updated_train.reset_index()

Продажи товара в предыдущие ```window``` месяцев



In [8]:
def sales_prev_year(df, lag):
  df['sales_month_prev_year'] = df.date_block_num - lag
  # df[df['sales_month_prev_year'] >= 0]
  shifted = df.copy()
  shifted= shifted[['shop_id', 'item_id', 'date_block_num', 'item_cnt_day']]
  shifted = shifted.rename(columns={'date_block_num' :'sales_month_prev_year'})
  df = df.merge(shifted, how='left', on=['shop_id', 'item_id', 'sales_month_prev_year'])
  # df = df[df['item_cnt_day_y'].notna()]
  df = df.rename(columns={"item_cnt_day_y":"sales_" + str(lag) +"_month_ago", "item_cnt_day_x":"item_cnt_day"})
  # df = df[df['date_block_num'] > lag]
  return df

In [9]:
updated_train = sales_prev_year(updated_train,1)
window = 6
for i in range(2, window):
  print(i)
  updated_train = sales_prev_year(updated_train, i)

2
3
4
5


Удаление первых ```window ``` месяцев



In [11]:
updated_train = updated_train[updated_train['date_block_num'] > window - 2]

Добавление общих продаж магазинов и товаров

In [12]:
updated_train = updated_train.merge(average_sales_shops, how='left', on='shop_id')
updated_train = updated_train.rename(columns={"item_cnt_day_x" : "item_cnt_day", "item_cnt_day_y":"shop_sales"})
updated_train = updated_train.merge(average_sales_items, how='left', on='item_id')
updated_train = updated_train.rename(columns={"item_cnt_day_x" : "item_cnt_day", "item_cnt_day_y": "item_sales"})

In [14]:
updated_train = updated_train.drop(columns=['sales_month_prev_year', 'item_id', 'shop_id'])

Добавление номера месяца

In [15]:
updated_train['date_block_num'] = updated_train['date_block_num'] % 12

In [17]:
target = updated_train['item_cnt_day']
X = updated_train.drop(columns='item_cnt_day')

In [18]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

In [20]:
ll = LinearRegression()
ll.fit(X_train, y_train)
predicted = ll.predict(X_test)
mean_absolute_error(predicted, y_test)

0.2992066132714348